# 1. Importación de paquetes

In [1]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [2]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [3]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [4]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [5]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [6]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [7]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [8]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
4231,25.38,25.44,24.94,25.56,0,0,0,0,0.07,0.04,0.06,0.06,370,-0.253846,0,0,0
8286,25.94,26.13,25.81,25.94,148,234,177,24,0.11,0.13,0.08,0.10,800,1.515385,0,0,1
1651,25.31,25.31,24.88,25.75,0,0,0,0,0.06,0.05,0.06,0.06,370,0.000000,0,0,0
1960,25.19,25.19,24.69,25.56,0,0,0,0,0.08,0.05,0.06,0.06,360,0.000000,0,0,0
3565,26.25,26.19,25.88,26.38,148,234,185,10,0.07,0.23,0.15,0.07,925,3.700000,0,1,1


In [9]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
7049,25.38,25.38,25.00,25.88,0,0,0,0,0.07,0.06,0.06,0.10,355,0.000000,0,0,0
1617,25.31,25.31,24.88,25.75,0,0,0,0,0.06,0.05,0.06,0.06,370,0.000000,0,0,0
8625,25.50,25.50,25.19,25.63,0,0,0,0,0.07,0.05,0.06,0.09,415,-1.465385,0,0,0
4923,25.19,25.19,24.56,25.50,0,0,0,0,0.06,0.05,0.06,0.06,360,-0.019231,0,0,0
8997,25.25,25.31,24.94,25.38,0,0,0,0,0.07,0.05,0.05,0.09,345,0.000000,0,0,0


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [10]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [11]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
4231,25.38,25.44,24.94,25.56,0,0,0,0,0.07,0.04,0.06,0.06,370,-0.253846,0,0
8286,25.94,26.13,25.81,25.94,148,234,177,24,0.11,0.13,0.08,0.10,800,1.515385,0,0
1651,25.31,25.31,24.88,25.75,0,0,0,0,0.06,0.05,0.06,0.06,370,0.000000,0,0
1960,25.19,25.19,24.69,25.56,0,0,0,0,0.08,0.05,0.06,0.06,360,0.000000,0,0
3565,26.25,26.19,25.88,26.38,148,234,185,10,0.07,0.23,0.15,0.07,925,3.700000,0,1


In [12]:
y.head()

4231    0
8286    1
1651    0
1960    0
3565    1
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [13]:
training.train_LR_codecarbon(X,y)
training.train_RF_codecarbon(X,y)
training.train_SVC_codecarbon(X,y)

[codecarbon INFO @ 17:20:16] [setup] RAM Tracking...
[codecarbon INFO @ 17:20:16] [setup] GPU Tracking...
[codecarbon INFO @ 17:20:16] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:20:16] [setup] CPU Tracking...
[codecarbon WARNING @ 17:20:16] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:20:18] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:18] >>> Tracker's metadata:
[codecarbon INFO @ 17:20:18]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 17:20:18]   Python version: 3.10.4
[codecarbon INFO @ 17:20:18]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:20:18]   CPU count: 6
[codecarbon INFO @ 17:20:18]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:18]   GPU count: 1
[codecarbon INFO @ 17:20:18]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 17:20:18] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 1.1702253391987465e-07 kg


[codecarbon INFO @ 17:20:20] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:20] >>> Tracker's metadata:
[codecarbon INFO @ 17:20:20]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 17:20:20]   Python version: 3.10.4
[codecarbon INFO @ 17:20:20]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:20:20]   CPU count: 6
[codecarbon INFO @ 17:20:20]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:20]   GPU count: 1
[codecarbon INFO @ 17:20:20]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 17:20:20] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 17:20:20] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 17:20:20] Energy consumed for all CPUs : 0.000002 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 17:20:20] 0.000002 kWh of electricity used since the begining.
[codecarbon INFO @ 17:20:20] [setup] 

Emissions: 3.5087832942067923e-07 kg


[codecarbon INFO @ 17:20:22] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:22] >>> Tracker's metadata:
[codecarbon INFO @ 17:20:22]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 17:20:22]   Python version: 3.10.4
[codecarbon INFO @ 17:20:22]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:20:22]   CPU count: 6
[codecarbon INFO @ 17:20:22]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:22]   GPU count: 1
[codecarbon INFO @ 17:20:22]   GPU model: 1 x NVIDIA GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
[codecarbon INFO @ 17:20:23] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 17:20:23] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 17:20

Emissions: 8.521739336298613e-08 kg
